In [1]:
import scipy.sparse as sp
from gcn.utils import *

import random
import tensorflow as tf

In [2]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features

def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res

def adj_times_x(adj, x, adj_pow=1):
    """Multiplies (adj^adj_pow)*x."""
    for i in range(adj_pow):
        x = tf.sparse_tensor_dense_matmul(adj, x)
    return x

def mixhop_layer(x, sparse_adjacency, adjacency_powers, dim_per_power,
                 kernel_regularizer=None, layer_id=None, replica=None):
    """Constructs MixHop layer.
    Args:
    sparse_adjacency: Sparse tensor containing square and normalized adjacency
      matrix.
    adjacency_powers: list of integers containing powers of adjacency matrix.
    dim_per_power: List same size as `adjacency_powers`. Each power will emit
      the corresponding dimensions.
    layer_id: If given, will be used to name the layer
    """
    #
    replica = replica or 0
    layer_id = layer_id or 0
    segments = []
    for p, dim in zip(adjacency_powers, dim_per_power):
        net_p = adj_times_x(sparse_adjacency, x, p)
        layer = tf.layers.dense(net_p, units=dim,
                                kernel_regularizer=kernel_regularizer,
                                activation=None, use_bias=False)
        segments.append(layer)
    return tf.concat(segments, axis=1)


def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    loss = -tf.reduce_sum(labels*tf.log(tf.nn.softmax(preds)+1e-7), axis=1)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)


In [3]:
class Mixhop:
    def __init__(self, n_nodes, f_dimension, nb_classes, lr, prob, lam):
        
        self.X = tf.placeholder(tf.float32, shape=[n_nodes, f_dimension], name='X')
        self.y = tf.placeholder('float32', name='y')
        self.mask = tf.placeholder('float32', name='Mask')
        self.adj = tf.sparse_placeholder(tf.float32, name='ADJ')
        self.dropout = tf.placeholder(tf.float32)
        self.prob = prob
        self.nodes = n_nodes
        
        self.L = mixhop_layer(self.X, self.adj, [0, 1, 2], [17, 22, 21],
                 kernel_regularizer=None, layer_id=0)
        self.L = mixhop_layer(self.L, self.adj, [0, 1, 2], [17, 22, 21],
                         kernel_regularizer=None, layer_id=1)
        self.L = mixhop_layer(self.L, self.adj, [0, 1, 2], [20, 20, 20],
                         kernel_regularizer=None, layer_id=2)
        
        self.logits = tf.layers.dense(self.L, units=nb_classes, activation=None)
        self.loss(lr, lam)
        
    def loss(self, lr, lam):
        var = tf.trainable_variables()
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in var if 'bias' not in v.name]) * lam
        self.cost = masked_softmax_cross_entropy(self.logits, self.y, self.mask) + lossL2

      
        self.optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.trains = self.optimizer.minimize(self.cost)
        self.acc = masked_accuracy(self.logits, self.y, self.mask)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
        
    def accuracy(self, X, Y, mask, adj):
        """Get accuracy"""
        return self.sess.run(self.acc,
                             feed_dict={self.adj: adj, self.X: X, self.y: Y, self.mask: mask,
                                       self.dropout:1.0})


    def train(self, batch_xs, batch_ys, mask, adj):
        _ = self.sess.run(self.trains, feed_dict={ self.y: batch_ys, self.adj: adj
            ,self.X: batch_xs, self.mask: mask, self.dropout:self.prob})

In [4]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data('cora')
N, D=features.shape
adj_dense=(adj+sp.eye(adj.shape[0])).todense()
features=preprocess_features(features).todense()
adj_gcn=cheb_adj(adj)
adj=rownorm_adj(adj)
num_nonzero_feat = features[1].shape

In [5]:
tf.reset_default_graph()
model = Mixhop(N, D, y_train.shape[1], 0.01, 0.5, 0.0005)
best = 0
for epoch in range(301):
    model.train(features, y_train, train_mask, adj_gcn)
    test_acc = model.accuracy(features, y_test, test_mask, adj_gcn)
    train_acc = model.accuracy(features, y_train, train_mask, adj_gcn)
    if best > test_acc:
        pass
    else:
        best = test_acc
print(best)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
0.80799997
